# Modelling

### Trainning

In [2]:
%sh pwd

/databricks/driver

In [3]:
%sh ls /dbfs/FileStore/tables/



all_vars_for_analysis.csv
all_vars_for_zeroinf_analysis.csv
Feature_Importances_predicting_deaths-d1408.csv
Final_Features_for_analysis-2c971.csv
Final_Features_for_analysis.csv
health.csv
health_factors_behab.csv
health_factors.csv
Health_Professional_Shortage_Areas__Primary_Care__by_County-c0119.csv
Lockdown_Join_with_normaliztion_factor-ed399.csv
merged.csv
States_Lock_down_Lengh-8d3af.csv
testing_by_state-971a2.csv
Train_X.csv
us_counties.csv
us_counties-e349f.csv
X_test.csv
X_train.csv
y_test.csv
y_train.csv

In [4]:
from sklearn import tree

from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,BayesianRidge,Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from statsmodels.api import OLS
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import numpy as np
import pandas as pd
#import pydotplus
from re import sub
import scipy.stats as sci
import seaborn as sns
 
# Set visualization prefrences 
sns.set(font_scale=1.5, style="darkgrid")

In [5]:
# File location and type
file_location = "/FileStore/tables/merged.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

## import all_vars_for_zeroinf_analysis  file 
# The applied options are for CSV files. For other file types, these will be ignored.
merged = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [6]:
display(merged)

_c0,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths,POP_ESTIMATE_2018,total_state_pop,Active.Physicians.per.100.000.Population..2018..AAMC.,Total.Active.Patient.Care.Physicians.per.100.000.Population..2018..AAMC.,Active.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.,Active.Patient.Care.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.,Active.General.Surgeons.per.100.000.Population..2018..AAMC.,Active.Patient.Care.General.Surgeons.per.100.000.Population..2018..AAMC.,Percentage.of.Active.Physicians.Who.Are.Female..2018..AAMC.,Percentage.of.Active.Physicians.Who.Are.Intertiol.Medical.Graduates..IMGs...2018..AAMC.,Percentage.of.Active.Physicians.Who.Are.Age.60.or.Older..2018..AAMC.,MD.and.DO.Student.Enrollment.per.100.000.Population..AY.2018.2019..AAMC.,Student.Enrollment.at.Public.MD.and.DO.Schools.per.100.000.Population..AY.2018.2019..AAMC.,Percentage.Change.in.Student.Enrollment.at.MD.and.DO.Schools..2008.2018..AAMC.,Percentage.of.MD.Students.Matriculating.In.State..AY.2018.2019..AAMC.,Total.Residents.Fellows.in.ACGME.Programs.per.100.000.Population.as.of.December.31..2018..AAMC.,Total.Residents.Fellows.in.Primary.Care.ACGME.Programs.per.100.000.Population.as.of.Dec..31..2018..AAMC.,Percentage.of.Residents.in.ACGME.Programs.Who.Are.IMGs.as.of.December.31..2018..AAMC.,Ratio.of.Residents.and.Fellows..GME..to.Medical.Students..UME...AY.2017.2018..AAMC.,Percent.Change.in.Residents.and.Fellows.in.ACGME.Accredited.Programs..2008.2018..AAMC.,Percentage.of.Physicians.Retained.in.State.from.Undergraduate.Medical.Education..UME...2018..AAMC.,All.Specialties..AAMC.,State.Local.Government.hospital.beds.per.1000.people..2019.,Non.profit.hospital.beds.per.1000.people..2019.,For.profit.hospital.beds.per.1000.people..2019.,Total.hospital.beds.per.1000.people..2019.,Total.nurse.practitioners..2019.,Total.physician.assistants..2019.,Total.Hospitals..2019.,Total.Primary.Care.Physicians..2019.,Surgery.specialists..2019.,Emergency.Medicine.specialists..2019.,Total.Specialist.Physicians..2019.,ICU.Beds,pop_fraction,Length.of.Life.rank,Quality.of.Life.rank,Health.Behaviors.rank,Clinical.Care.rank,Social...Economic.Factors.rank,Physical.Environment.rank,Adult.smoking.percentage,Adult.obesity.percentage,Excessive.drinking.percentage,Population.per.sq.mile,House.per.sq.mile,Total.Number.of.COVID.19.Tests.with.Results,Share.of.Tests.with.Positive.COVID.19.Results,Number.of.Tests.with.Results.per.1.000.Population
1,5/14/2020,Autauga,Alabama,1001,93,4,93,4,null,null,55601,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.50%,18.10%,34.40%,49.1,22.8,136.80%,82.10%,29.9,11.9,21.90%,0.6,18.40%,48.40%,120.7374364,1.4,0.8,0.9,3.1,28.85913663,6.085785611,1.148905321,66.69338512,8.576976356,5.289514596,72.14215392,6,0.00568765,5,11,9,13,7,53,18,33,15,91.8,37.2,136372,0.077853,27.8
2,5/14/2020,Baldwin,Alabama,1003,231,7,231,7,null,null,218022,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.50%,18.10%,34.40%,49.1,22.8,136.80%,82.10%,29.9,11.9,21.90%,0.6,18.40%,48.40%,473.4342433,1.4,0.8,0.9,3.1,113.1621137,23.86351236,4.505074295,261.5173326,33.63194078,20.74118364,282.882982,51,0.022302348,3,2,5,6,3,5,17,31,18,114.6,65.5,136372,0.077853,27.8
3,5/14/2020,Barbour,Alabama,1005,69,1,69,1,null,null,24881,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.50%,18.10%,34.40%,49.1,22.8,136.80%,82.10%,29.9,11.9,21.90%,0.6,18.40%,48.40%,54.02903104,1.4,0.8,0.9,3.1,12.91423137,2.72334008,0.514125884,29.84475306,3.838127888,2.36701521,32.28303325,5,0.002545178,23,55,54,41,60,28,22,42,13,31.0,13.4,136372,0.077853,27.8
4,5/14/2020,Bibb,Alabama,1007,46,1,46,1,null,null,22400,9775742,217.1,196.7,77.2,71.2,7.6,6.9,28.50%,18.10%,34.40%,49.1,22.8,136.80%,82.10%,29.9,11.9,21.90%,0.6,18.40%,48.40%,48.64154557,1.4,0.8,0.9,3.1,11.62649342,2.4517832,0.462860006,26.86879421,3.455410341,2.130989136,29.06394215,0,0.002291386,49,14,32,18,43,22,19,38,16,36.8,14.4,136372,0.077853,27.8
5,5/14/2020,Blount,Alabama,1009,45,0,45,0,null,null,57840,9775

In [7]:
pandas_merged = merged.toPandas()

In [8]:
pandas_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997 entries, 0 to 2996
Data columns (total 60 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 _c0 2997 non-null int32 
 1 date 2997 non-null object 
 2 county 2997 non-null object 
 3 state 2997 non-null object 
 4 fips 2966 non-null float64
 5 cases 2997 non-null int32 
 6 deaths 2920 non-null float64
 7 confirmed_cases 2422 non-null float64
 8 confirmed_deaths 2099 non-null float64
 9 probable_cases 273 non-null float64
 10 probable_deaths 142 non-null float64
 11 POP_ESTIMATE_2018 2890 non-null float64
 12 total_state_pop 2890 non-null float64
 13 Active.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 14 Total.Active.Patient.Care.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 15 Active.Primary.Care.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 16 Active.Patient.Care.Primary.Care.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 17 Active.General.Surgeons.per.100.000.Population..2018..AAMC. 2890 non-null float64
 18 Active.Patient.Care.General.Surgeons.per.100.000.Population..2018..AAMC. 2890 non-null float64
 19 Percentage.of.Active.Physicians.Who.Are.Female..2018..AAMC. 2890 non-null object 
 20 Percentage.of.Active.Physicians.Who.Are.Intertiol.Medical.Graduates..IMGs...2018..AAMC. 2890 non-null object 
 21 Percentage.of.Active.Physicians.Who.Are.Age.60.or.Older..2018..AAMC. 2890 non-null object 
 22 MD.and.DO.Student.Enrollment.per.100.000.Population..AY.2018.2019..AAMC. 2821 non-null float64
 23 Student.Enrollment.at.Public.MD.and.DO.Schools.per.100.000.Population..AY.2018.2019..AAMC. 2689 non-null float64
 24 Percentage.Change.in.Student.Enrollment.at.MD.and.DO.Schools..2008.2018..AAMC. 2774 non-null object 
 25 Percentage.of.MD.Students.Matriculating.In.State..AY.2018.2019..AAMC. 2772 non-null object 
 26 Total.Residents.Fellows.in.ACGME.Programs.per.100.000.Population.as.of.December.31..2018..AAMC. 2890 non-null float64
 27 Total.Residents.Fellows.in.Primary.Care.ACGME.Programs.per.100.000.Population.as.of.Dec..31..2018..AAMC. 2890 non-null float64
 28 Percentage.of.Residents.in.ACGME.Programs.Who.Are.IMGs.as.of.December.31..2018..AAMC. 2890 non-null object 
 29 Ratio.of.Residents.and.Fellows..GME..to.Medical.Students..UME...AY.2017.2018..AAMC. 2821 non-null float64
 30 Percent.Change.in.Residents.and.Fellows.in.ACGME.Accredited.Programs..2008.2018..AAMC. 2890 non-null object 
 31 Percentage.of.Physicians.Retained.in.State.from.Undergraduate.Medical.Education..UME...2018..AAMC. 2788 non-null object 
 32 All.Specialties..AAMC. 2890 non-null float64
 33 State.Local.Government.hospital.beds.per.1000.people..2019. 2773 non-null float64
 34 Non.profit.hospital.beds.per.1000.people..2019. 2890 non-null float64
 35 For.profit.hospital.beds.per.1000.people..2019. 2723 non-null float64
 36 Total.hospital.beds.per.1000.people..2019. 2890 non-null float64
 37 Total.nurse.practitioners..2019. 2890 non-null float64
 38 Total.physician.assistants..2019. 2890 non-null float64
 39 Total.Hospitals..2019. 2890 non-null float64
 40 Total.Primary.Care.Physicians..2019. 2890 non-null float64
 41 Surgery.specialists..2019. 2890 non-null float64
 42 Emergency.Medicine.specialists..2019. 2890 non-null float64
 43 Total.Specialist.Physicians..2019. 2890 non-null float64
 44 ICU.Beds 2890 non-null float64
 45 pop_fraction 2890 non-null float64
 46 Length.of.Life.rank 2890 non-null object 
 47 Quality.of.Life.rank 2890 non-null object 
 48 Health.Behaviors.rank 2890 non-null object 
 49 Clinical.Care.rank 2890 non-null object 
 50 Social...Economic.Factors.rank 2890 non-null object 
 51 Physical.Environment.rank 2890 non-null object 
 52 Adult.smoking.percentage 2890 non-null float64
 53 Adult.obesity.percentage 2890 non-null float64
 54 Excessive.drinking.percentage 2890 non-null float64
 55 Population.per.sq.mile 2965 non-null float64
 56 House.per.sq.mile 2965 non-null float

In [9]:
#pandas_merged['cases'].unique()
#pandas_merged['deaths'].unique()

In [10]:
# Drop rows with unknown counties.
pandas_merged = pandas_merged[pandas_merged['county']!= 'Unknown']
pandas_merged.reset_index(drop=True)

,_c0,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths,POP_ESTIMATE_2018,total_state_pop,Active.Physicians.per.100.000.Population..2018..AAMC.,Total.Active.Patient.Care.Physicians.per.100.000.Population..2018..AAMC.,Active.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.,Active.Patient.Care.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.,Active.General.Surgeons.per.100.000.Population..2018..AAMC.,Active.Patient.Care.General.Surgeons.per.100.000.Population..2018..AAMC.,Percentage.of.Active.Physicians.Who.Are.Female..2018..AAMC.,Percentage.of.Active.Physicians.Who.Are.Intertiol.Medical.Graduates..IMGs...2018..AAMC.,Percentage.of.Active.Physicians.Who.Are.Age.60.or.Older..2018..AAMC.,MD.and.DO.Student.Enrollment.per.100.000.Population..AY.2018.2019..AAMC.,Student.Enrollment.at.Public.MD.and.DO.Schools.per.100.000.Population..AY.2018.2019..AAMC.,Percentage.Change.in.Student.Enrollment.at.MD.and.DO.Schools..2008.2018..AAMC.,Percentage.of.MD.Students.Matriculating.In.State..AY.2018.2019..AAMC.,Total.Residents.Fellows.in.ACGME.Programs.per.100.000.Population.as.of.December.31..2018..AAMC.,Total.Residents.Fellows.in.Primary.Care.ACGME.Programs.per.100.000.Population.as.of.Dec..31..2018..AAMC.,Percentage.of.Residents.in.ACGME.Programs.Who.Are.IMGs.as.of.December.31..2018..AAMC.,Ratio.of.Residents.and.Fellows..GME..to.Medical.Students..UME...AY.2017.2018..AAMC.,Percent.Change.in.Residents.and.Fellows.in.ACGME.Accredited.Programs..2008.2018..AAMC.,Percentage.of.Physicians.Retained.in.State.from.Undergraduate.Medical.Education..UME...2018..AAMC.,All.Specialties..AAMC.,State.Local.Government.hospital.beds.per.1000.people..2019.,Non.profit.hospital.beds.per.1000.people..2019.,For.profit.hospital.beds.per.1000.people..2019.,Total.hospital.beds.per.1000.people..2019.,Total.nurse.practitioners..2019.,Total.physician.assistants..2019.,Total.Hospitals..2019.,Total.Primary.Care.Physicians..2019.,Surgery.specialists..2019.,Emergency.Medicine.specialists..2019.,Total.Specialist.Physicians..2019.,ICU.Beds,pop_fraction,Length.of.Life.rank,Quality.of.Life.rank,Health.Behaviors.rank,Clinical.Care.rank,Social...Economic.Factors.rank,Physical.Environment.rank,Adult.smoking.percentage,Adult.obesity.percentage,Excessive.drinking.percentage,Population.per.sq.mile,House.per.sq.mile,Total.Number.of.COVID.19.Tests.with.Results,Share.of.Tests.with.Positive.COVID.19.Results,Number.of.Tests.with.Results.per.1.000.Population
0,1,5/14/2020,Autauga,Alabama,1001.0,93,4.0,93.0,4.0,NaN,NaN,55601.0,9775742.0,217.1,196.7,77.2,71.2,7.6,6.9,28.50%,18.10%,34.40%,49.1,22.8,136.80%,82.10%,29.9,11.9,21.90%,0.6,18.40%,48.40%,120.737436,1.4,0.8,0.9,3.1,28.859137,6.085786,1.148905,66.693385,8.576976,5.289515,72.142154,6.0,0.005688,5,11,9,13,7,53,18.0,33.0,15.0,91.8,37.2,136372.0,0.077853,27.8
1,2,5/14/2020,Baldwin,Alabama,1003.0,231,7.0,231.0,7.0,NaN,NaN,218022.0,9775742.0,217.1,196.7,77.2,71.2,7.6,6.9,28.50%,18.10%,34.40%,49.1,22.8,136.80%,82.10%,29.9,11.9,21.90%,0.6,18.40%,48.40%,473.434243,1.4,0.8,0.9,3.1,113.162114,23.863512,4.505074,261.517333,33.631941,20.741184,282.882982,51.0,0.022302,3,2,5,6,3,5,17.0,31.0,18.0,114.6,65.5,136372.0,0.077853,27.8
2,3,5/14/2020,Barbour,Alabama,1005.0,69,1.0,69.0,1.0,NaN,NaN,24881.0,9775742.0,217.1,196.7,77.2,71.2,7.6,6.9,28.50%,18.10%,34.40%,49.1,22.8,136.80%,82.10%,29.9,11.9,21.90%,0.6,18.40%,48.40%,54.029031,1.4,0.8,0.9,3.1,12.914231,2.723340,0.514126,29.844753,3.838128,2.367015,32.283033,5.0,0.002545,23,55,54,41,60,28,22.0,42.0,13.0,31.0,13.4,136372.0,0.077853,27.8
3,4,5/14/2020,Bibb,Alabama,1007.0,46,1.0,46.0,1.0,NaN,NaN,22400.0,9775742.0,217.1,196.7,77.2,71.2,7.6,6.9,28.50%,18.10%,34.40%,49.1,22.8,136.80%,82.10%,29.9,11.9,21.90%,0.6,18.40%,48.40%,48.641546,1.4,0.8,0.9,3.1,11.626493,2.451783,0.462860,26.868794,3.455410,2.130989,29.063942,0.0,0.002291,49,14,32,18,43,22,19.0,38.0,16.0,36.8,14.4,136372.0,0.077853,27.8
4,5,5/14/2020,Blount,Alabama,1009.0,45,0.0,45.0,0.0

In [11]:
# Drop fips & indices columns
pandas_merged.drop(['fips','_c0'],inplace=True,axis=1)

In [12]:
# Drop Total.Number.of.COVID.19.Tests.with.Results, as it is identical to Number.of.Tests.with.Results.per.1.000.Population (but not normalized)
pandas_merged.drop('Total.Number.of.COVID.19.Tests.with.Results',inplace=True, axis=1)

In [13]:
# Check dates availiable
pandas_merged['date'].unique()

Out[148]: array(['5/14/2020'], dtype=object)

In [14]:
# Drop dates, as there's only one value.
pandas_merged.drop(['date'], axis=1, inplace=True);

In [15]:
# Remove rows with Nulls for cases or deaths (y variables)
y_na_n = pandas_merged.shape[0]
pandas_merged = pandas_merged[pandas_merged['cases'].notna()]
pandas_merged = pandas_merged[pandas_merged['deaths'].notna()]
y_na_n = y_na_n - pandas_merged.shape[0]
print('# Rows removed: ',str(y_na_n))

# Rows removed: 76

In [16]:
# Remove columns with 25% or more NAs
na_cols = pandas_merged.columns[pandas_merged.isna().sum() > pandas_merged.shape[0]*.25]
print('Columns removed:\n',na_cols)
pandas_merged = pandas_merged[pandas_merged.columns[~pandas_merged.columns.isin(na_cols)]]

Columns removed:
 Index(['confirmed_deaths', 'probable_cases', 'probable_deaths'], dtype='object')

In [17]:
# Check difference between cases and confirmed cases
print('NAs in cases: ', pandas_merged['cases'].isna().sum())
print('NAs in confirmed cases: ', pandas_merged['confirmed_cases'].isna().sum())

print('% different cases: ', sum(pandas_merged['cases']!=pandas_merged['confirmed_cases'])/pandas_merged.shape[0]*100)


NAs in cases: 0
NAs in confirmed cases: 494
% different cases: 22.23374827109267

In [18]:
# Remove confirmed cases, as it has many NAs
pandas_merged.drop('confirmed_cases', inplace=True, axis=1)

In [19]:
# Check maximal NA % of remaining features
round(np.max(pandas_merged.isna().sum())/pandas_merged.shape[0]*100,2)

Out[154]: 7.02

In [20]:
non_features = ['county','state','POP_ESTIMATE_2018','total_state_pop', 
                'pop_fraction']

to_be_normalzied_features = ['cases','deaths','All.Specialties..AAMC.',
                             'Total.nurse.practitioners..2019.',
                             'Total.physician.assistants..2019.',
                             'Total.Hospitals..2019.',
                             'Total.Primary.Care.Physicians..2019.',
                             'Surgery.specialists..2019.',
                             'Emergency.Medicine.specialists..2019.',
                             'Total.Specialist.Physicians..2019.',
                             'ICU.Beds',
                             ]

in_percentage = ['Percentage_active_Physicians_femele_2018_AAMC',
                 'Percentage_active_Physicians_Intertiol_Medical_Graduates_IMGs_2018_AAMC',
                 'Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC',
                 'Percentage_active_Physicians_Age_60_older_2018_AAMC',
                 'Percentage_change_student_Enrollment_MD_DO_Schools_2008_2018_AAMC',
                 'Percentage_MD_student_Matriculating_In_State_AY_20018_2019_AAMC',
                 'Percentage_Residents_In_ACGME_Programs_IMGs_December_31_2018_AAMC',
                 'Percentage_Residents_and_Fellows_Programs_In_ACGME_Accredited_Programs_2008_2018_AAMC',
                 'Percentage_Physicians_Retained_in_State_From_Undergraduate_Accredited_Medical_Education_UME_2018_AAMC'
                 ]


In [21]:
pandas_merged = pandas_merged.rename(columns={
 'Percentage.of.Active.Physicians.Who.Are.Female..2018..AAMC.': 'Percentage_active_Physicians_femele_2018_AAMC',
  'Percentage.of.Active.Physicians.Who.Are.Intertiol.Medical.Graduates..IMGs...2018..AAMC.': 'Percentage_active_Physicians_Intertiol_Medical_Graduates_IMGs_2018_AAMC',
  'Percentage.of.Active.Physicians.Who.Are.International.Medical.Graduates..IMGs...2018..AAMC.':'Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC', 
 'Percentage.of.Active.Physicians.Who.Are.Age.60.or.Older..2018..AAMC.': 'Percentage_active_Physicians_Age_60_older_2018_AAMC',
 'Percentage.Change.in.Student.Enrollment.at.MD.and.DO.Schools..2008.2018..AAMC.': 'Percentage_change_student_Enrollment_MD_DO_Schools_2008_2018_AAMC',
 'Percentage.of.MD.Students.Matriculating.In.State..AY.2018.2019..AAMC.': 'Percentage_MD_student_Matriculating_In_State_AY_20018_2019_AAMC',
 'Percentage.of.Residents.in.ACGME.Programs.Who.Are.IMGs.as.of.December.31..2018..AAMC.': 'Percentage_Residents_In_ACGME_Programs_IMGs_December_31_2018_AAMC', 
 'Percent.Change.in.Residents.and.Fellows.in.ACGME.Accredited.Programs..2008.2018..AAMC.': 'Percentage_Residents_and_Fellows_Programs_In_ACGME_Accredited_Programs_2008_2018_AAMC', 
 'Percentage.of.Physicians.Retained.in.State.from.Undergraduate.Medical.Education..UME...2018..AAMC.': 'Percentage_Physicians_Retained_in_State_From_Undergraduate_Accredited_Medical_Education_UME_2018_AAMC'})

In [23]:
pandas_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2892 entries, 0 to 2996
Data columns (total 52 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 county 2892 non-null object 
 1 state 2892 non-null object 
 2 cases 2892 non-null int32 
 3 deaths 2892 non-null float64
 4 POP_ESTIMATE_2018 2890 non-null float64
 5 total_state_pop 2890 non-null float64
 6 Active.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 7 Total.Active.Patient.Care.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 8 Active.Primary.Care.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 9 Active.Patient.Care.Primary.Care.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 10 Active.General.Surgeons.per.100.000.Population..2018..AAMC. 2890 non-null float64
 11 Active.Patient.Care.General.Surgeons.per.100.000.Population..2018..AAMC. 2890 non-null float64
 12 Percentage_active_Physicians_femele_2018_AAMC 0 non-null float64
 13 Percentage_active_Physicians_Intertiol_Medical_Graduates_IMGs_2018_AAMC 0 non-null float64
 14 Percentage_active_Physicians_Age_60_older_2018_AAMC 2890 non-null object 
 15 MD.and.DO.Student.Enrollment.per.100.000.Population..AY.2018.2019..AAMC. 2821 non-null float64
 16 Student.Enrollment.at.Public.MD.and.DO.Schools.per.100.000.Population..AY.2018.2019..AAMC. 2689 non-null float64
 17 Percentage_change_student_Enrollment_MD_DO_Schools_2008_2018_AAMC 2774 non-null object 
 18 Percentage_MD_student_Matriculating_In_State_AY_20018_2019_AAMC 2772 non-null object 
 19 Total.Residents.Fellows.in.ACGME.Programs.per.100.000.Population.as.of.December.31..2018..AAMC. 2890 non-null float64
 20 Total.Residents.Fellows.in.Primary.Care.ACGME.Programs.per.100.000.Population.as.of.Dec..31..2018..AAMC. 2890 non-null float64
 21 Percentage_Residents_In_ACGME_Programs_IMGs_December_31_2018_AAMC 2890 non-null object 
 22 Ratio.of.Residents.and.Fellows..GME..to.Medical.Students..UME...AY.2017.2018..AAMC. 2821 non-null float64
 23 Percentage_Residents_and_Fellows_Programs_In_ACGME_Accredited_Programs_2008_2018_AAMC 2890 non-null object 
 24 Percentage_Physicians_Retained_in_State_From_Undergraduate_Accredited_Medical_Education_UME_2018_AAMC 2788 non-null object 
 25 All.Specialties..AAMC. 2890 non-null float64
 26 State.Local.Government.hospital.beds.per.1000.people..2019. 2773 non-null float64
 27 Non.profit.hospital.beds.per.1000.people..2019. 2890 non-null float64
 28 For.profit.hospital.beds.per.1000.people..2019. 2723 non-null float64
 29 Total.hospital.beds.per.1000.people..2019. 2890 non-null float64
 30 Total.nurse.practitioners..2019. 2890 non-null float64
 31 Total.physician.assistants..2019. 2890 non-null float64
 32 Total.Hospitals..2019. 2890 non-null float64
 33 Total.Primary.Care.Physicians..2019. 2890 non-null float64
 34 Surgery.specialists..2019. 2890 non-null float64
 35 Emergency.Medicine.specialists..2019. 2890 non-null float64
 36 Total.Specialist.Physicians..2019. 2890 non-null float64
 37 ICU.Beds 2890 non-null float64
 38 pop_fraction 2890 non-null float64
 39 Length.of.Life.rank 2890 non-null object 
 40 Quality.of.Life.rank 2890 non-null object 
 41 Health.Behaviors.rank 2890 non-null object 
 42 Clinical.Care.rank 2890 non-null object 
 43 Social...Economic.Factors.rank 2890 non-null object 
 44 Physical.Environment.rank 2890 non-null object 
 45 Adult.smoking.percentage 2890 non-null float64
 46 Adult.obesity.percentage 2890 non-null float64
 47 Excessive.drinking.percentage 2890 non-null float64
 48 Population.per.sq.mile 2889 non-null float64
 49 House.per.sq.mile 2889 non-null float64
 50 Share.of.Tests.with.Positive.COVID.19.Results 2756 non-null float64
 51 Number.of.Tests.with.Results.per.1.000.Population 2892 non-null float64
dtypes: float64(37), int32(1), object(14)
memory usage: 1.2+ MB

In [24]:
pandas_merged['Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC'] = pandas_merged['Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC'].apply(lambda x: sub('%','',str(x))).astype(float)

--------------------------------------------------------------------------- 
 KeyError Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc (self, key, method, tolerance) 
 2645 try : 
 -> 2646 return self . _engine . get_loc ( key ) 
 2647 except KeyError : 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 KeyError : 'Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC'

During handling of the above exception, another exception occurred:

 KeyError Traceback (most recent call last)
 <command-3303225798117847> in <module> 
 ----> 1 pandas_merged [ 'Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC' ] = pandas_merged [ 'Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC' ] . apply ( lambda x : sub ( '%' , '' , str ( x ) ) ) . astype ( float ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/frame.py in __getitem__ (self, key) 
 2798 if self . columns . nlevels > 1 : 
 2799 return self . _getitem_multilevel ( key ) 
 -> 2800 indexer = self . columns . get_loc ( key ) 
 2801 if is_integer ( indexer ) : 
 2802 indexer = [ indexer ] 

 /databricks/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc (self, key, method, tolerance) 
 2646 return self . _engine . get_loc ( key ) 
 2647 except KeyError : 
 -> 2648 return self . _engine . get_loc ( self . _maybe_cast_indexer ( key ) ) 
 2649 indexer = self . get_indexer ( [ key ] , method = method , tolerance = tolerance ) 
 2650 if indexer . ndim > 1 or indexer . size > 1 : 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 KeyError : 'Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC'

In [25]:
# Convert percentiles into floats
for col_name in in_percentage:
  pandas_merged[col_name] = pandas_merged[col_name].apply(lambda x: sub('%','',str(x))).astype(float)
  #pandas_merged[col_name] = (pandas_merged[col_name].str.split()).apply(lambda x: float(x[0].replace(',', '')))
  #pandas_merged[col_name] = pandas_merged[col_name].str.replace(',', ".").astype(float)
  #pandas_merged[col_name] = pd.to_numeric( pandas_merged[col_name],errors='coerce')

--------------------------------------------------------------------------- 
 KeyError Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc (self, key, method, tolerance) 
 2645 try : 
 -> 2646 return self . _engine . get_loc ( key ) 
 2647 except KeyError : 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 KeyError : 'Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC'

During handling of the above exception, another exception occurred:

 KeyError Traceback (most recent call last)
 <command-2187800200639426> in <module> 
 1 # Convert percentiles into floats 
 2 for col_name in in_percentage : 
 ----> 3 pandas_merged [ col_name ] = pandas_merged [ col_name ] . apply ( lambda x : sub ( '%' , '' , str ( x ) ) ) . astype ( float ) 
 4 #pandas_merged[col_name] = (pandas_merged[col_name].str.split()).apply(lambda x: float(x[0].replace(',', ''))) 
 5 #pandas_merged[col_name] = pandas_merged[col_name].str.replace(',', ".").astype(float) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/frame.py in __getitem__ (self, key) 
 2798 if self . columns . nlevels > 1 : 
 2799 return self . _getitem_multilevel ( key ) 
 -> 2800 indexer = self . columns . get_loc ( key ) 
 2801 if is_integer ( indexer ) : 
 2802 indexer = [ indexer ] 

 /databricks/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc (self, key, method, tolerance) 
 2646 return self . _engine . get_loc ( key ) 
 2647 except KeyError : 
 -> 2648 return self . _engine . get_loc ( self . _maybe_cast_indexer ( key ) ) 
 2649 indexer = self . get_indexer ( [ key ] , method = method , tolerance = tolerance ) 
 2650 if indexer . ndim > 1 or indexer . size > 1 : 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 KeyError : 'Percentage_active_Physicians_International_Medical_Graduates_2018_AAMC'

In [26]:
# Normalize features
for feature in to_be_normalzied_features:
  pandas_merged[feature] = pandas_merged[feature]*1000/pandas_merged['POP_ESTIMATE_2018']

In [27]:
# Check all variables, make sure all are numerical.
pandas_merged[pandas_merged.columns[~pandas_merged.columns.isin(non_features)]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2892 entries, 0 to 2996
Data columns (total 47 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 cases 2890 non-null float64
 1 deaths 2890 non-null float64
 2 Active.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 3 Total.Active.Patient.Care.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 4 Active.Primary.Care.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 5 Active.Patient.Care.Primary.Care.Physicians.per.100.000.Population..2018..AAMC. 2890 non-null float64
 6 Active.General.Surgeons.per.100.000.Population..2018..AAMC. 2890 non-null float64
 7 Active.Patient.Care.General.Surgeons.per.100.000.Population..2018..AAMC. 2890 non-null float64
 8 Percentage_active_Physicians_femele_2018_AAMC 0 non-null float64
 9 Percentage_active_Physicians_Intertiol_Medical_Graduates_IMGs_2018_AAMC 0 non-null float64
 10 Percentage_active_Physicians_Age_60_older_2018_AAMC 2890 non-null object 
 11 MD.and.DO.Student.Enrollment.per.100.000.Population..AY.2018.2019..AAMC. 2821 non-null float64
 12 Student.Enrollment.at.Public.MD.and.DO.Schools.per.100.000.Population..AY.2018.2019..AAMC. 2689 non-null float64
 13 Percentage_change_student_Enrollment_MD_DO_Schools_2008_2018_AAMC 2774 non-null object 
 14 Percentage_MD_student_Matriculating_In_State_AY_20018_2019_AAMC 2772 non-null object 
 15 Total.Residents.Fellows.in.ACGME.Programs.per.100.000.Population.as.of.December.31..2018..AAMC. 2890 non-null float64
 16 Total.Residents.Fellows.in.Primary.Care.ACGME.Programs.per.100.000.Population.as.of.Dec..31..2018..AAMC. 2890 non-null float64
 17 Percentage_Residents_In_ACGME_Programs_IMGs_December_31_2018_AAMC 2890 non-null object 
 18 Ratio.of.Residents.and.Fellows..GME..to.Medical.Students..UME...AY.2017.2018..AAMC. 2821 non-null float64
 19 Percentage_Residents_and_Fellows_Programs_In_ACGME_Accredited_Programs_2008_2018_AAMC 2890 non-null object 
 20 Percentage_Physicians_Retained_in_State_From_Undergraduate_Accredited_Medical_Education_UME_2018_AAMC 2788 non-null object 
 21 All.Specialties..AAMC. 2890 non-null float64
 22 State.Local.Government.hospital.beds.per.1000.people..2019. 2773 non-null float64
 23 Non.profit.hospital.beds.per.1000.people..2019. 2890 non-null float64
 24 For.profit.hospital.beds.per.1000.people..2019. 2723 non-null float64
 25 Total.hospital.beds.per.1000.people..2019. 2890 non-null float64
 26 Total.nurse.practitioners..2019. 2890 non-null float64
 27 Total.physician.assistants..2019. 2890 non-null float64
 28 Total.Hospitals..2019. 2890 non-null float64
 29 Total.Primary.Care.Physicians..2019. 2890 non-null float64
 30 Surgery.specialists..2019. 2890 non-null float64
 31 Emergency.Medicine.specialists..2019. 2890 non-null float64
 32 Total.Specialist.Physicians..2019. 2890 non-null float64
 33 ICU.Beds 2890 non-null float64
 34 Length.of.Life.rank 2890 non-null object 
 35 Quality.of.Life.rank 2890 non-null object 
 36 Health.Behaviors.rank 2890 non-null object 
 37 Clinical.Care.rank 2890 non-null object 
 38 Social...Economic.Factors.rank 2890 non-null object 
 39 Physical.Environment.rank 2890 non-null object 
 40 Adult.smoking.percentage 2890 non-null float64
 41 Adult.obesity.percentage 2890 non-null float64
 42 Excessive.drinking.percentage 2890 non-null float64
 43 Population.per.sq.mile 2889 non-null float64
 44 House.per.sq.mile 2889 non-null float64
 45 Share.of.Tests.with.Positive.COVID.19.Results 2756 non-null float64
 46 Number.of.Tests.with.Results.per.1.000.Population 2892 non-null float64
dtypes: float64(35), object(12)
memory usage: 1.1+ MB

In [28]:
pandas_merged[pandas_merged.columns[~pandas_merged.columns.isin(non_features)]] = pandas_merged[pandas_merged.columns[~pandas_merged.columns.isin(non_features)]].fillna(pandas_merged[pandas_merged.columns[~pandas_merged.columns.isin(non_features)]].mean())

## Feature selection: via random tree gini importance

In [30]:
X = pandas_merged[pandas_merged.columns[~pandas_merged.columns.isin(non_features)]].drop(['cases','deaths'], axis=1) # Exclude deaths & cases from X
y = pandas_merged['deaths']

In [31]:
# Check number of features
print('# features: ', X.shape[1])

# features: 45

In [32]:
# Verify there are no nulls.
X.isnull().any().any()

Out[172]: True

In [33]:
# Split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, pandas_merged['deaths'], test_size=0.25, random_state=10)


In [34]:
# train random forst
n_trees= 1000

rf = RandomForestRegressor(n_estimators=n_trees, oob_score=True, random_state=10)

rf.fit(X_train, y_train)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-3303225798117855> in <module> 
 4 rf = RandomForestRegressor ( n_estimators = n_trees , oob_score = True , random_state = 10 ) 
 5 
 ----> 6 rf . fit ( X_train , y_train ) 

 /databricks/python/lib/python3.7/site-packages/sklearn/ensemble/_forest.py in fit (self, X, y, sample_weight) 
 293 """
 294 # Validate or convert input data 
 --> 295 X = check_array ( X , accept_sparse = "csc" , dtype = DTYPE ) 
 296 y = check_array ( y , accept_sparse = 'csc' , ensure_2d = False , dtype = None ) 
 297 if sample_weight is not None : 

 /databricks/python/lib/python3.7/site-packages/sklearn/utils/validation.py in check_array (array, accept_sparse, accept_large_sparse, dtype, order, copy, force_all_finite, ensure_2d, allow_nd, ensure_min_samples, ensure_min_features, warn_on_dtype, estimator) 
 529 array = array . astype ( dtype , casting = "unsafe" , copy = False ) 
 530 else : 
 --> 531 array = np . asarray ( array , order = order , dtype = dtype ) 
 532 except ComplexWarning : 
 533 raise ValueError("Complex data not supported\n"

 /databricks/python/lib/python3.7/site-packages/numpy/core/_asarray.py in asarray (a, dtype, order) 
 83 
 84 """
 ---> 85 return array ( a , dtype , copy = False , order = order ) 
 86 
 87 

 ValueError : could not convert string to float: '30.40%'